In [ ]:
import cv2
import numpy as np
import keras
from keras.models import load_model
import os

# Load the trained model
model = load_model('vgg16_model.h5')

# Mapping of class indices to class labels
class_labels = [name for name in sorted(os.listdir('AugmentedData')) if not name.startswith('.')]

print(class_labels)

In [ ]:
# Set up the virtual board
canvas_size = (500, 800)
canvas = np.ones((canvas_size[0], canvas_size[1], 3), dtype=np.uint8) * 255
window_name = 'Doodle Guesser'
cv2.namedWindow(window_name)

# Variables for drawing functionality
drawing = False
prev_point = None
drawing_color = (0, 0, 0)  # Default drawing color is black

# Function to predict the doodle
def predict_doodle(doodle):
    doodle = cv2.resize(doodle, (224, 224))  # Resize doodle to match model input size
    doodle = cv2.cvtColor(doodle, cv2.COLOR_GRAY2BGR)  # Convert grayscale to 3 channels
    doodle = np.expand_dims(doodle, axis=0)  # Add batch dimension
    doodle = doodle / 255.0  # Normalize the doodle image
    prediction = model.predict(doodle)
    predicted_class_index = np.argmax(prediction)
    predicted_class_label = class_labels[predicted_class_index]
    confidence = prediction[0][predicted_class_index] * 100
    return predicted_class_label, confidence

# Function to handle mouse events
def draw(event, x, y, flags, param):
    global canvas, drawing, prev_point

    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        prev_point = (x, y)

    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            cv2.line(canvas, prev_point, (x, y), drawing_color, 4)
            prev_point = (x, y)

    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False

# Register the mouse event callback
cv2.setMouseCallback(window_name, draw)

# Main loop for drawing and prediction
while True:
    cv2.imshow(window_name, canvas)
    key = cv2.waitKey(1) & 0xFF

    # Change the drawing color
    if key == ord('b'):  # Blue color
        drawing_color = (255, 0, 0)
    elif key == ord('g'):  # Green color
        drawing_color = (0, 255, 0)
    elif key == ord('r'):  # Red color
        drawing_color = (0, 0, 255)
    elif key == ord('k'):  # Black color
        drawing_color = (0, 0, 0)

    # Reset the canvas if 'c' key is pressed
    if key == ord('c'):
        canvas = np.ones((canvas_size[0], canvas_size[1], 3), dtype=np.uint8) * 255

    # Predict the doodle if 's' key is pressed
    if key == ord('s'):
        doodle = cv2.cvtColor(canvas, cv2.COLOR_BGR2GRAY)
        predicted_class, confidence = predict_doodle(doodle)
        print('Predicted class:', predicted_class)
        print('Confidence:', confidence)

    # Exit the loop if 'q' key is pressed
    if key == ord('q'):
        break

# Clean up
cv2.destroyAllWindows()